In [ ]:
#r "BoSSSpad.dll"
// #r ".\binaries\XNSEC.dll"
// #r "C:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
// #r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\XNSEC.dll"
//#r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\XNSEC.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
Init();

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Unable to open database: 
primary path: d:\bosss_db_Natconv
No alternative paths specified.
IOException caught while opening database #0 at 'd:\bosss_db_Natconv': Unable to open database - all given paths either don't exist or are ruled out by the machine filter on this machine with name pcmit30 .
Unable to open database: 
primary path: \\hpccluster\hpccluster-scratch\gutierrez\NatConvStudyTEST
No alternative paths specified.
IOException caught while opening database #1 at '\\hpccluster\hpccluster-scratch\gutierrez\NatConvStudyTEST': Unable to open database - all given paths either don't exist or are ruled out by the machine filter on this machine with name pcmit30 .
Unable to open database: 
primary path: \\hpccluster\hpccluster-scratch\gutierrez\bosss_db_Natconv
No alternative paths specified.
IOException caught while opening database #2 at '\\hpccluster\hpccluste

Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in C:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 170
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in C:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 97
   at Submission#79.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
using BoSSS.Application.XNSEC;

In [ ]:
BoSSSshell.WorkflowMgm.Init("CounterFlowFlame");

Project name is set to 'CounterFlowFlame'.
NotSupportedException caught during creation/opening of default database: `AllowedDatabasesPaths` not specified, unable to create Database (should be specified in ~/.BoSSS/etc/BatchProcessorConfig.json)..


In [ ]:
var myBatch = BoSSSshell.GetDefaultQueue();
var myDb = myBatch.CreateOrOpenCompatibleDatabase("CounterFlowFlame");
// var myDb = OpenOrCreateDatabase(@"C:\Databases\CounterFlowFlame");


Opening existing database 'C:\Databases\CounterFlowFlame'.


In [ ]:
int counter = 0;
int[] mults = new int[] { 2, 5 , 11 };
Dictionary<int,Plot2Ddata[,] > PlotTableS = new Dictionary<int, Plot2Ddata[,]>();

foreach (int mult in mults) {
    counter++;

    var sess = myDb.Sessions.Where(sess => sess.Name.Contains("mult" + mult)).FirstOrDefault(); // Search the actual session
    if(sess == null)
        continue;
        
    var timestep_FullChem = sess.Timesteps.Last();
    double eps = 1e-4;
    double xleft = 0 + eps;
    double xright = 1.0 - eps;
    int nCells = 80; // Number of points to be picked
    double[] _xNodes = GenericBlas.Linspace(xleft, xright, nCells + 1);
    string[] varnames = new string[] { "VelocityX", "Temperature", "MassFraction0", "MassFraction1", "MassFraction2", "MassFraction3" };

    Dictionary<string, double[]> Results_FullComb = new Dictionary<string, double[]>();
    MultidimensionalArray[] Results_FullComb2 = new MultidimensionalArray[varnames.Length];
    Dictionary<string, double> ReferenceValuesDict = new Dictionary<string, double>();// Re-dimenzionalise variables

    ReferenceValuesDict.Add("VelocityX", Convert.ToDouble(sess.KeysAndQueries["uRef"]));
    ReferenceValuesDict.Add("Temperature", Convert.ToDouble(sess.KeysAndQueries["TRef"]));
    ReferenceValuesDict.Add("MassFraction0", 1.0);
    ReferenceValuesDict.Add("MassFraction1", 1.0);
    ReferenceValuesDict.Add("MassFraction2", 1.0);
    ReferenceValuesDict.Add("MassFraction3", 1.0);
    ReferenceValuesDict.Add("Length", Convert.ToDouble(sess.KeysAndQueries["LRef"]));

    
    //Pick and save results from the BoSSS calculation 
    int cnt = -1;
    double[] _xNodesDim = new double[_xNodes.Length];
    foreach (var varname in varnames) {
        cnt++;
        double[] Yvalues = new double[nCells + 1];
        var field = timestep_FullChem.Fields.Where(Fi => (Fi.Identification == varname)).Last();
        for (int n = 0; n < _xNodes.Length; n++) {
            _xNodesDim[n] = _xNodes[n] * ReferenceValuesDict["Length"]; // Redimensionalise the x coordinate
            double pickedVal = field.ProbeAt(new double[] { _xNodes[n], 0.00000001 });    // Pick value along the central axis        
            Yvalues[n] = pickedVal * ReferenceValuesDict[varname];             // Re.dimenzionalise varliables and store it in array
        }
        var xyVals = MultidimensionalArray.Create(_xNodes.Length, _xNodes.Length);
        for (int i = 0; i < _xNodesDim.Length; i++) {
            xyVals[i, 0] = _xNodesDim[i];
            xyVals[i, 1] = Yvalues[i];
        }
        Results_FullComb2[cnt] = xyVals;
    }


    // Read results from the Matlab calculation of the 1D problem.
    string[] VariablesForPlot = new string[] { "VelocityX", "Temperature", "MF0", "MF1", "MF2", "MF3" };
    MultidimensionalArray[] ReferenceData = new MultidimensionalArray[VariablesForPlot.Length];
    string CurrentDocDir = Directory.GetCurrentDirectory();
    //Load data from Matlab
    for (int i = 0; i < VariablesForPlot.Length; i++) {
        string path = String.Concat(CurrentDocDir, @"\ML" + mult + "_", VariablesForPlot[i], ".txt");
        ReferenceData[i] = IMatrixExtensions.LoadFromTextFile(path);
    }

    

    Plot2Ddata[,] PlotTable = new Plot2Ddata[2, 3];
    int cnt2 = -1;
    for (int iCol = 0; iCol < 3; iCol++) {
        for (int iRow = 0; iRow < 2; iRow++) {
            var plot = new Plot2Ddata();
            var allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
            cnt2++;
            // BoSSS data
            var fmt = new PlotFormat();
            fmt.Style = Styles.Lines;
            fmt.LineColor = allColors[cnt2 % allColors.Length];
            fmt.DashType = DashTypes.Solid;
            
            plot.AddDataGroup( "BoSSS",
                Results_FullComb2[cnt2].ExtractSubArrayShallow(-1, 0).To1DArray(),
                Results_FullComb2[cnt2].ExtractSubArrayShallow(-1, 1).To1DArray(),
                fmt);

            // Matlab data
            fmt = new PlotFormat();
            fmt.Style = Styles.Lines;
            fmt.LineColor = allColors[cnt2 % allColors.Length];
            fmt.DashType = DashTypes.Dashed;
            plot.AddDataGroup( "Matlab",
            ReferenceData[cnt2].ExtractSubArrayShallow(-1, 0).To1DArray(),
            ReferenceData[cnt2].ExtractSubArrayShallow(-1, 1).To1DArray(),
            fmt);
            plot.Title = varnames[cnt2];
            plot.TitleFont = 20;
            plot.Xlabel = "x (m)";
            
            if(iRow == 0 && iCol == 0)
                plot.Ylabel = "velocity (m/s)";
            if(iRow == 1 && iCol == 0)
                plot.Ylabel = "Temperature (K)";
            PlotTable[iRow, iCol] = plot;
        }
    }
    PlotTableS.Add(counter,PlotTable);

}

## Results for low inlet velocity case

In [ ]:
var gp = PlotTableS[1].ToGnuplot();
gp.PlotSVG(xRes:1500,yRes:600)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"Left reverse 
set key font ",16"Left reverse 
set key font ",16"Left reverse 
set key font ",16"Left reverse 
set key font ",16"Left reverse 
set key font ",16"Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.14 
 
 
 
 
 -0.12 
 
 
 
 
 -0.1 
 
 
 
 
 -0.08 
 
 
 
 
 -0.06 
 
 
 
 
 -0.04 
 
 
 
 
 -0.02 
 
 
 
 
 0 
 
 
 
 
 0.02 
 
 
 
 
 0.04 
 
 
 
 
 0.06 
 
 
 
 
 0 
 
 
 
 
 0.005 
 
 
 
 
 0.01 
 
 
 
 
 0.015 
 
 
 
 
 0.02 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 velocity (m/s) 
 
 
 
 
 x (m) 
 
 
 
 
 VelocityX 
 
 
 BoSSS 
 
 
 BoSSS 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M305.7,108.1 L359.1,108.1 M85.2,95.8 L89.7,95.9 L94.2,96.3 L98.7,96.8 L103.2,97.5 L107.7,98.4
 L112.2,99.3 L116.7,100.1 L121.2,100.9 L125.7,101.5 L130.2,102.0 L134.7,102.4 L139.2,102.7 L143.7,103.2
 L148.2,104.0 L152.7,105.1 L157.2,106.6 L161.7,108.7 L166.2,111.3 L170.7,114.5 L175.2,118.2 L179.7,122.5
 L184.2,127.2 L188.7,132.4 L193.2,138.0 L197.7,143.9 L202.2,150.0 L206.7,156.4 L211.2,163.0 L215.7,169.7
 L220.2,176.5 L224.7,183.3 L229.2,189.9 L233.7,195.5 L238.2,198.7 L242.7,199.4 L247.2,198.7 L251.7,197.3
 L256.2,195.4 L260.7,193.3 L265.2,190.8 L269.6,188.2 L274.1,185.5 L278.6,182.9 L283.1,180.4 L287.6,178.2
 L292.1,176.5 L296.6,175.3 L301.1,174.8 L305.6,175.1 L310.1,176.1 L314.6,177.7 L319.1,179.8 L323.6,182.1
 L328.1,184.6 L332.6,187.3 L337.1,189.9 L341.6,192.5 L346.1,195.1 L350.6,197.6 L355.1,200.1 L359.6,202.4
 L364.1,204.7 L368.6,206.9 L373.1,209.0 L377.6,211.1 L382.1,213.0 L386.6,214.7 L391.1,216.4 L395.6,218.0
 L400.1,219.4 L404.6,220.8 L409.1,221.9 L413.6,223.0 L418.1,223.9 L422.6,224.7 L427.1,225.3 L431.6,225.9
 L436.1,226.2 L440.6,226.4 L445.1,226.5 '/> 
 
 Matlab 
 
 
 Matlab 
 
 
 
	<path stroke='rgb(255, 0, 0)' stroke-dasharray='2.5,4.0' d='M305.7,132.1 L359.1,132.1 M85.2,95.8 L87.0,95.8 L88.8,95.8 L90.6,95.8 L92.4,95.9 L94.2,96.0
 L96.1,96.1 L97.9,96.3 L99.7,96.4 L101.5,96.6 L103.3,96.8 L105.1,97.0 L106.9,97.1 L108.7,97.3
 L110.5,97.4 L112.3,97.5 L114.1,97.6 L115.9,97.6 L117.8,97.7 L119.6,97.7 L121.4,97.6 L123.2,97.6
 L125.0,97.5 L126.8,97.5 L128.6,97.4 L130.4,97.4 L132.2,97.4 L134.0,97.4 L135.8,97.5 L137.6,97.6
 L139.5,97.8 L141.3,98.1 L143.1,98.4 L144.9,98.8 L146.7,99.3 L148.5,99.9 L150.3,100.6 L152.1,101.4
 L153.9,102.3 L155.7,103.3 L157.5,104.4 L159.4,105.6 L161.2,106.9 L163.0,108.3 L164.8,109.8 L166.6,111.4
 L168.4,113.1 L170.2,114.9 L172.0,116.8 L173.8,118.8 L175.6,120.9 L177.4,123.0 L179.2,125.3 L181.1,127.6
 L182.9,130.0 L184.7,132.4 L186.5,134.9 L188.3,137.5 L190.1,140.2 L191.9,142.9 L193.7,145.6 L195.5,148.4
 L197.3,151.2 L199.1,154.1 L200.9,157.0 L202.8,160.0 L204.6,162.9 L206.4,165.9 L208.2,168.9 L210.0,172.0
 L211.8,175.0 L213.6,178.0 L215.4,181.1 L217.2,184.2 L219.0,187.2 L220.8,190.3 L222.6,193.3 L224.5,196.1
 L226.3,198.7 L228.1,201.0 L229.9,202.8 L231.7,204.0 L233.5,204.8 L235.3,205.2 L237.1,205.3 L238.9,205.1
 L240.7,204.8 L242.5,204.3 L244.4,203.7 L246.2,202.9 L248.0,202.1 L249.8,201.2 L251.6,200.2 L253.4,199.1
 L255.2,197.9 L257.0,196.7 L258.8,195.4 L260.6,194.1 L262.4,192.7 L264.2,191.3 L266.1,189.8 L267.9,188.3
 L269.7,186.8 L271.5,185.3 L273.3,183.9 L275.1,182.5 L276.9,181.1 L278.7,179.8 L280.5,178.6 L282.3,177.5
 L284.1,176.6 L285.9,175.8 L287.8,175.1 L289.6,174.6 L291.4,174.3 L293.2,174.2 L295.0,174.2 L296.8,174.4
 L298.6,174.8 L300.4,175.3 L302.2,175.9 L304.0,176.6 L305.8,177.4 L307.7,178.2 L309.5,179.2 L311.3,180.1
 L313.1,181.1 L314.9,182.1 L316.7,183.2 L318.5,184.2 L320.3,185.3 L322.1,186.3 L323.9,187.4 L325.7,188.5
 L327.5,189.5 L329.4,190.5 L331.2,191.6 L333.0,192.6 L334.8,193.6 L336.6,194.6 L338.4,195.6 L340.2,196.6
 L342.0,197.5 L343.8,198.5 L345.6,199.4 L347.4,200.3 L349.2,201.3 L351.1,202.2 L352.9,203.0 L354.7,203.9
 L356.5,204.8 L358.3,205.6 L360.1,206.4 L361.9,207.2 L363.7,208.0 L365.5,208

## Results for mid inlet velocity case

In [ ]:
var gp = PlotTableS[2].ToGnuplot();
gp.PlotSVG(xRes:1500,yRes:600)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"Left reverse 
set key font ",16"Left reverse 
set key font ",16"Left reverse 
set key font ",16"Left reverse 
set key font ",16"Left reverse 
set key font ",16"Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.35 
 
 
 
 
 -0.3 
 
 
 
 
 -0.25 
 
 
 
 
 -0.2 
 
 
 
 
 -0.15 
 
 
 
 
 -0.1 
 
 
 
 
 -0.05 
 
 
 
 
 0 
 
 
 
 
 0.05 
 
 
 
 
 0.1 
 
 
 
 
 0.15 
 
 
 
 
 0 
 
 
 
 
 0.005 
 
 
 
 
 0.01 
 
 
 
 
 0.015 
 
 
 
 
 0.02 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 velocity (m/s) 
 
 
 
 
 x (m) 
 
 
 
 
 VelocityX 
 
 
 BoSSS 
 
 
 BoSSS 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M305.7,108.1 L359.1,108.1 M85.2,95.8 L89.7,96.0 L94.2,96.4 L98.7,97.0 L103.2,97.9 L107.7,99.1
 L112.2,100.5 L116.7,102.1 L121.2,103.8 L125.7,105.8 L130.2,107.8 L134.7,109.7 L139.2,111.3 L143.7,112.6
 L148.2,113.5 L152.7,114.1 L157.2,114.8 L161.7,115.9 L166.2,117.6 L170.7,120.0 L175.2,123.3 L179.7,127.3
 L184.2,132.0 L188.7,137.2 L193.2,142.9 L197.7,148.9 L202.2,155.2 L206.7,161.5 L211.2,167.5 L215.7,171.6
 L220.2,172.7 L224.7,171.8 L229.2,170.1 L233.7,167.9 L238.2,165.4 L242.7,163.0 L247.2,160.8 L251.7,159.0
 L256.2,158.1 L260.7,158.3 L265.2,159.4 L269.6,161.2 L274.1,163.5 L278.6,166.0 L283.1,168.6 L287.6,171.3
 L292.1,173.9 L296.6,176.5 L301.1,179.1 L305.6,181.6 L310.1,184.1 L314.6,186.5 L319.1,188.9 L323.6,191.3
 L328.1,193.6 L332.6,195.8 L337.1,198.0 L341.6,200.2 L346.1,202.2 L350.6,204.3 L355.1,206.2 L359.6,208.1
 L364.1,209.9 L368.6,211.6 L373.1,213.2 L377.6,214.8 L382.1,216.2 L386.6,217.6 L391.1,218.9 L395.6,220.1
 L400.1,221.2 L404.6,222.2 L409.1,223.1 L413.6,223.9 L418.1,224.6 L422.6,225.2 L427.1,225.6 L431.6,226.0
 L436.1,226.3 L440.6,226.4 L445.1,226.5 '/> 
 
 Matlab 
 
 
 Matlab 
 
 
 
	<path stroke='rgb(255, 0, 0)' stroke-dasharray='2.5,4.0' d='M305.7,132.1 L359.1,132.1 M85.2,95.8 L87.0,95.9 L88.8,95.9 L90.6,96.1 L92.4,96.2 L94.2,96.4
 L96.1,96.7 L97.9,97.0 L99.7,97.3 L101.5,97.7 L103.3,98.1 L105.1,98.6 L106.9,99.1 L108.7,99.6
 L110.5,100.2 L112.3,100.8 L114.1,101.5 L115.9,102.1 L117.8,102.9 L119.6,103.6 L121.4,104.3 L123.2,105.1
 L125.0,105.9 L126.8,106.6 L128.6,107.3 L130.4,108.0 L132.2,108.7 L134.0,109.2 L135.8,109.7 L137.6,110.1
 L139.5,110.5 L141.3,110.7 L143.1,110.9 L144.9,111.1 L146.7,111.2 L148.5,111.3 L150.3,111.4 L152.1,111.6
 L153.9,111.8 L155.7,112.2 L157.5,112.6 L159.4,113.2 L161.2,113.8 L163.0,114.6 L164.8,115.6 L166.6,116.7
 L168.4,117.9 L170.2,119.2 L172.0,120.7 L173.8,122.4 L175.6,124.1 L177.4,126.0 L179.2,127.9 L181.1,130.0
 L182.9,132.2 L184.7,134.5 L186.5,136.8 L188.3,139.2 L190.1,141.7 L191.9,144.3 L193.7,146.9 L195.5,149.5
 L197.3,152.2 L199.1,154.9 L200.9,157.7 L202.8,160.4 L204.6,163.2 L206.4,165.9 L208.2,168.6 L210.0,171.1
 L211.8,173.2 L213.6,174.7 L215.4,175.6 L217.2,176.1 L219.0,176.1 L220.8,175.9 L222.6,175.3 L224.5,174.7
 L226.3,173.8 L228.1,172.9 L229.9,171.8 L231.7,170.7 L233.5,169.5 L235.3,168.2 L237.1,166.9 L238.9,165.5
 L240.7,164.2 L242.5,162.9 L244.4,161.7 L246.2,160.6 L248.0,159.6 L249.8,158.8 L251.6,158.3 L253.4,158.0
 L255.2,157.9 L257.0,158.1 L258.8,158.4 L260.6,159.0 L262.4,159.7 L264.2,160.6 L266.1,161.5 L267.9,162.5
 L269.7,163.5 L271.5,164.6 L273.3,165.7 L275.1,166.8 L276.9,167.9 L278.7,169.0 L280.5,170.1 L282.3,171.2
 L284.1,172.2 L285.9,173.3 L287.8,174.4 L289.6,175.5 L291.4,176.5 L293.2,177.6 L295.0,178.6 L296.8,179.7
 L298.6,180.7 L300.4,181.7 L302.2,182.7 L304.0,183.7 L305.8,184.7 L307.7,185.7 L309.5,186.7 L311.3,187.7
 L313.1,188.6 L314.9,189.6 L316.7,190.6 L318.5,191.5 L320.3,192.4 L322.1,193.3 L323.9,194.2 L325.7,195.1
 L327.5,196.0 L329.4,196.9 L331.2,197.8 L333.0,198.6 L334.8,199.5 L336.6,200.3 L338.4,201.1 L340.2,201.9
 L342.0,202.7 L343.8,203.5 L345.6,204.3 L347.4,205.1 L349.2,205.8 L351.1,206.6 L352.9,207.3 L354.7,208.0
 L356.5,208.7 L358.3,209.4 L360.1,210.1 L361.9,210.8 L36

## Results for high inlet velocity case

In [ ]:
var gp = PlotTableS[3].ToGnuplot();
gp.PlotSVG(xRes:1500,yRes:600)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"Left reverse 
set key font ",16"Left reverse 
set key font ",16"Left reverse 
set key font ",16"Left reverse 
set key font ",16"Left reverse 
set key font ",16"Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.7 
 
 
 
 
 -0.6 
 
 
 
 
 -0.5 
 
 
 
 
 -0.4 
 
 
 
 
 -0.3 
 
 
 
 
 -0.2 
 
 
 
 
 -0.1 
 
 
 
 
 0 
 
 
 
 
 0.1 
 
 
 
 
 0.2 
 
 
 
 
 0.3 
 
 
 
 
 0 
 
 
 
 
 0.005 
 
 
 
 
 0.01 
 
 
 
 
 0.015 
 
 
 
 
 0.02 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 velocity (m/s) 
 
 
 
 
 x (m) 
 
 
 
 
 VelocityX 
 
 
 BoSSS 
 
 
 BoSSS 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M305.7,108.1 L359.1,108.1 M76.9,92.1 L81.5,92.3 L86.1,92.7 L90.7,93.4 L95.3,94.3 L99.9,95.6
 L104.5,97.0 L109.1,98.7 L113.7,100.6 L118.4,102.8 L123.0,105.1 L127.6,107.6 L132.2,110.2 L136.8,112.9
 L141.4,115.4 L146.0,117.4 L150.6,118.7 L155.2,119.6 L159.8,120.7 L164.4,122.8 L169.0,126.0 L173.6,130.5
 L178.2,135.8 L182.8,141.9 L187.4,148.3 L192.0,154.9 L196.6,159.5 L201.2,160.3 L205.8,158.7 L210.4,156.5
 L215.0,154.1 L219.6,152.0 L224.2,151.0 L228.8,151.4 L233.4,153.2 L238.0,155.7 L242.6,158.4 L247.2,161.3
 L251.8,164.1 L256.4,166.9 L261.0,169.7 L265.6,172.5 L270.2,175.2 L274.8,177.9 L279.4,180.6 L284.0,183.2
 L288.6,185.8 L293.2,188.4 L297.8,190.9 L302.4,193.3 L307.0,195.8 L311.6,198.1 L316.2,200.5 L320.8,202.7
 L325.4,205.0 L330.0,207.1 L334.6,209.2 L339.2,211.3 L343.8,213.2 L348.4,215.1 L353.0,217.0 L357.6,218.7
 L362.2,220.4 L366.8,222.0 L371.4,223.5 L376.0,225.0 L380.6,226.4 L385.2,227.6 L389.8,228.8 L394.4,229.9
 L399.0,231.0 L403.6,231.9 L408.3,232.7 L412.9,233.4 L417.5,234.1 L422.1,234.6 L426.7,235.1 L431.3,235.4
 L435.9,235.7 L440.5,235.8 L445.1,235.9 '/> 
 
 Matlab 
 
 
 Matlab 
 
 
 
	<path stroke='rgb(255, 0, 0)' stroke-dasharray='2.5,4.0' d='M305.7,132.1 L359.1,132.1 M76.9,92.1 L78.8,92.2 L80.6,92.2 L82.5,92.3 L84.3,92.5 L86.2,92.7
 L88.0,92.9 L89.9,93.2 L91.7,93.6 L93.6,93.9 L95.4,94.4 L97.3,94.8 L99.1,95.3 L101.0,95.9
 L102.8,96.4 L104.7,97.1 L106.5,97.7 L108.4,98.4 L110.2,99.1 L112.1,99.9 L113.9,100.7 L115.8,101.6
 L117.6,102.4 L119.5,103.3 L121.3,104.3 L123.2,105.3 L125.0,106.2 L126.9,107.3 L128.7,108.3 L130.6,109.4
 L132.4,110.4 L134.3,111.5 L136.1,112.5 L138.0,113.5 L139.8,114.4 L141.7,115.2 L143.5,115.9 L145.4,116.5
 L147.2,116.9 L149.1,117.1 L150.9,117.4 L152.8,117.6 L154.6,117.8 L156.5,118.1 L158.3,118.6 L160.2,119.2
 L162.0,120.0 L163.9,121.1 L165.7,122.3 L167.6,123.7 L169.4,125.3 L171.3,127.1 L173.1,129.0 L175.0,131.2
 L176.8,133.4 L178.7,135.8 L180.5,138.3 L182.4,140.9 L184.2,143.6 L186.1,146.3 L187.9,149.1 L189.8,151.9
 L191.6,154.7 L193.5,157.5 L195.3,160.0 L197.2,161.7 L199.0,162.7 L200.9,163.1 L202.7,162.9 L204.6,162.3
 L206.4,161.5 L208.3,160.5 L210.1,159.4 L212.0,158.1 L213.8,156.8 L215.7,155.5 L217.5,154.2 L219.4,153.0
 L221.2,152.0 L223.1,151.3 L224.9,150.9 L226.8,150.9 L228.6,151.2 L230.5,151.8 L232.3,152.6 L234.2,153.6
 L236.0,154.7 L237.9,155.8 L239.7,157.0 L241.6,158.2 L243.4,159.4 L245.3,160.6 L247.1,161.8 L249.0,162.9
 L250.8,164.1 L252.7,165.3 L254.5,166.5 L256.4,167.6 L258.2,168.8 L260.1,170.0 L261.9,171.1 L263.8,172.3
 L265.6,173.4 L267.5,174.5 L269.3,175.7 L271.2,176.8 L273.0,177.9 L274.9,179.0 L276.7,180.1 L278.6,181.2
 L280.4,182.3 L282.3,183.4 L284.1,184.5 L286.0,185.5 L287.8,186.6 L289.7,187.6 L291.5,188.7 L293.4,189.7
 L295.2,190.7 L297.1,191.8 L298.9,192.8 L300.8,193.8 L302.6,194.8 L304.5,195.8 L306.3,196.7 L308.2,197.7
 L310.0,198.7 L311.9,199.6 L313.7,200.5 L315.6,201.5 L317.4,202.4 L319.3,203.3 L321.1,204.2 L323.0,205.1
 L324.8,206.0 L326.7,206.8 L328.5,207.7 L330.4,208.5 L332.2,209.4 L334.1,210.2 L335.9,211.0 L337.8,211.8
 L339.6,212.6 L341.5,213.4 L343.3,214.1 L345.2,214.9 L347.0,215.6 L348.9,216.4 L350.7,217.1 L352.6,217.8
 L354.4,218.5 L356.3,219.2 L358.1,219.8 L360.0,220.5 L361.8,221.1 L363.7,221